# Using 🤗 PEFT & bitsandbytes to finetune a LoRa checkpoint




In [16]:
# !pip install -q bitsandbytes datasets accelerate loralib
# !pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
!nvidia-smi -L

GPU 0: NVIDIA GeForce RTX 3090 (UUID: GPU-53573cab-fc5d-e8af-4423-124f958899fb)
GPU 1: NVIDIA GeForce RTX 3090 (UUID: GPU-c0e0a88f-c719-20b0-0c44-8b697299c276)
GPU 2: NVIDIA GeForce RTX 3090 (UUID: GPU-0aa8929e-1774-1dc0-5325-0dce1165c78c)
GPU 3: NVIDIA GeForce RTX 3090 (UUID: GPU-9f605028-04a0-7456-bed1-8e4e06d9d44a)
GPU 4: NVIDIA GeForce RTX 3090 (UUID: GPU-31d09e65-80fe-5f2a-c34a-17da33412ae0)
GPU 5: NVIDIA GeForce RTX 3090 (UUID: GPU-d1a545c6-1038-527d-ec65-40f66caf2807)
GPU 6: NVIDIA GeForce RTX 3090 (UUID: GPU-7db6b504-9087-7bac-ba63-f957f018e494)
GPU 7: NVIDIA GeForce RTX 3090 (UUID: GPU-65bf5d44-7fd3-2034-09cf-4db9a2746b2a)


In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="4"


### Setup the model

In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="4"
import torch
import torch.nn as nn
import bitsandbytes as bnb
import transformers
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
from transformers import AutoModelForSeq2SeqLM
from peft import LoraConfig, get_peft_model, TaskType

model_checkpoint = "t5-3b"
# model_checkpoint = "bigscience/mt0-xxl"
# model_checkpoint = "google/mt5-xl"


model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

from transformers import AutoModelForSeq2SeqLM

# huggingface hub model id
model_id = "philschmid/flan-t5-xxl-sharded-fp16"

# load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, load_in_8bit=True, device_map="auto")

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /home/user/ksharma/miniconda3/envs/ks_tars_thesis did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//matplotlib_inline.backend_inline'), PosixPath('module')}
  warn(msg)
/home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/cuda/lib64')}
  warn(msg)
/home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: No libcudart.so found! Install CUDA or the cudatoolkit package (anaconda)!
  warn(msg)


Loading checkpoint shards:   0%|          | 0/12 [00:00<?, ?it/s]

/home/user/ksharma/miniconda3/envs/ks_tars_thesis/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-3b automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


### Freezing the original weights


In [5]:
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, TaskType

# Define LoRA Config
lora_config = LoraConfig(
 r=16,
 lora_alpha=32,
 target_modules=["q", "v"],
 lora_dropout=0.05,
 bias="none",
 task_type=TaskType.SEQ_2_SEQ_LM
)
# prepare int-8 model for training
model = prepare_model_for_int8_training(model)

# add LoRA adaptor
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# trainable params: 18874368 || all params: 11154206720 || trainable%: 0.16921300163961817


/home/user/ksharma/.local/lib/python3.9/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


trainable params: 18,874,368 || all params: 11,154,206,720 || trainable%: 0.16921300163961817


## SJMM Data

In [6]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "4"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import datasets
import pandas as pd
# pd.set_option('display.width', 10000)
pd.set_option('display.max_colwidth', 1000)
# pd.set_option('display.max_columns', None)
data_path="/home/user/ksharma/ks_thesis/ma-thesis/pipeline_test/data_files/df_2000_rows.csv"

# data_df = pd.read_csv(data_path)



# train_ratio = 0.9
# val_ratio = 0.1
# test_ratio = 0.1
from sklearn.model_selection import train_test_split
# sjmm_data = pd.read_csv(data_path)
# sjmm_data['input'] = sjmm_data['input'].str.rstrip(' ->')
# fine_tuning_sjmm_data = sjmm_data[['input', 'output']]
# fine_tuning_sjmm_data['ind_skill'] = fine_tuning_sjmm_data['output'].str.findall(r'<SoftSkill>(.*?)</SoftSkill>')
# train_sjmm_data, test_sjmm_data = train_test_split(fine_tuning_sjmm_data, test_size=0.1, random_state=42)
# validation_sjmm_data, test_sjmm_data = train_test_split(test_sjmm_data, test_size=0.5, random_state=42)

data = pd.read_csv(data_path)
fine_tuning_data = data[['input', 'output']]
train_sjmm_data, test_sjmm_data = train_test_split(fine_tuning_data, test_size=0.1, random_state=42)

train_sjmm_data = datasets.Dataset.from_pandas(train_sjmm_data)
test_sjmm_data = datasets.Dataset.from_pandas(test_sjmm_data)

raw_datasets = datasets.DatasetDict({
    "train": train_sjmm_data,
    "test": test_sjmm_data
})

def preprocess_function(data):
    inputs = [noun_ellipsis for noun_ellipsis in data["input"]]
    # Replace special tokens in inputs
    # inputs = [input_text.replace('<SoftSkill_C>', '<SS_C>').replace('</SoftSkill_C>', '</SS_C>').replace('<SoftSkill>', '<SS>').replace('</SoftSkill>', '</SS>') for input_text in inputs]
    
    model_inputs = tokenizer(inputs, truncation=True, padding=True)

    # Setup the tokenizer for targets
    labels = tokenizer(data["output"], truncation=True, text_pair=data["input"], padding=True)
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs


tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["input", "output"])


Map:   0%|          | 0/1773 [00:00<?, ? examples/s]

Map:   0%|          | 0/198 [00:00<?, ? examples/s]

In [7]:
tokenized_datasets["train"]

Dataset({
    features: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1773
})

In [ ]:
# # Create a new Dataset object
# from datasets import Dataset, DatasetDict
# train_sjmm_dataset = Dataset.from_dict(train_sjmm_data)

# # Create a new DatasetDict object with the 'train' subset
# train_sjmm_dataset_dict = DatasetDict({'train': train_sjmm_dataset})

In [24]:
# train_sjmm_dataset_dict['train'][0]

In [25]:
# def merge_columns(example):
#     example["prediction"] = example["input"] + " == " + str(example["output"])
#     return example

# train_sjmm_dataset_dict['train'] = train_sjmm_dataset_dict['train'].map(merge_columns)
# train_sjmm_dataset_dict['train']["prediction"][:5]

In [26]:
# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
# tokenizer = transformers.AutoTokenizer.from_pretrained(model_checkpoint, model_max_length=512)
# train_sjmm_dataset_dict
# data_sjmm_dataset_dict = train_sjmm_dataset_dict.map(lambda samples: tokenizer(samples['prediction']), batched=True)

In [28]:
# data_sjmm_dataset_dict

### Training

In [8]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)


In [9]:
%env WANDB_PROJECT=final_graph

env: WANDB_PROJECT=final_graph


In [10]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

output_dir="lora-flan-t5-xxl"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
	auto_find_batch_size=True,
    learning_rate=1e-3, # higher learning rate
    # num_train_epochs=5,
    warmup_steps=100,
    max_steps=1740,
    logging_steps=1, 
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    run_name='new-flan-t5-xxl',
    # logging_steps=500,
    save_strategy="no",
    report_to="wandb",
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!


In [11]:
# train model
trainer.train()


/home/user/ksharma/miniconda3/envs/ks_tars_thesis/lib/python3.9/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: kartikey-sharma (ma-thesis). Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:321: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
1,23.323000
2,19.025400
3,29.114100
4,26.557400
5,25.355000
6,20.234200
7,28.008100
8,28.577700
9,25.930200
10,22.967800


TrainOutput(global_step=1740, training_loss=0.3534658139955459, metrics={'train_runtime': 6904.8537, 'train_samples_per_second': 1.008, 'train_steps_per_second': 0.252, 'total_flos': 1.0297477564622438e+17, 'train_loss': 0.3534658139955459, 'epoch': 3.92})

: 

## Share adapters on the 🤗 Hub

In [9]:
huggingface_token = "xxxxxxxxxxxxxxxxxx"
model.push_to_hub("Kartikey95/flan-t5-xxl-sharded-fp16",
                  use_auth_token=huggingface_token,
                  commit_message="basic training from philip model sharded",
                  private=True)

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/75.6M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Kartikey95/flan-t5-xxl-sharded-fp16/commit/7e48be8d5fe7df2b14b0c35a237b709fa7540468', commit_message='basic training from philip model sharded', commit_description='', oid='7e48be8d5fe7df2b14b0c35a237b709fa7540468', pr_url=None, pr_revision=None, pr_num=None)

## Load adapters from the Hub

In [10]:
from huggingface_hub import login
import os

# access_token = os.environ["xxxxxxxxxxxxxxxxxx"]
login(token=huggingface_token)


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/user/ksharma/.cache/huggingface/token
Login successful


In [48]:
# from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# quantization_config = BitsAndBytesConfig(llm_int8_enable_fp32_cpu_offload=True)

In [49]:
# device_map = {
#     "transformer.word_embeddings": 0,
#     "transformer.word_embeddings_layernorm": 0,
#     "lm_head": "cpu",
#     "transformer.h": 0,
#     "transformer.ln_f": 0,
# }

In [52]:
# model_8bit = AutoModelForCausalLM.from_pretrained(
#     "bigscience/bloom-1b7",
#     device_map=device_map,quantization_config=quantization_config,
# )

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="6"
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM

# peft_model_id = "Kartikey95/SJMM-bloom-7b1-lora-tagger"

peft_model_id = "Kartikey95/flan-t5-xxl-sharded-fp16"
# config = PeftConfig.from_pretrained(peft_model_id)
# model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')
# # model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True,device_map=device_map,quantization_config=quantization_config)
# tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# # Load the Lora model
# model = PeftModel.from_pretrained(model, peft_model_id)


config = PeftConfig.from_pretrained(peft_model_id)

# load base LLM model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path,  load_in_8bit=True,  device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id, device_map={"":0})
model.eval()

print("Peft model loaded")


Loading checkpoint shards:   0%|          | 0/12 [00:00<?, ?it/s]

Peft model loaded


## Inference

In [11]:
from sklearn.model_selection import train_test_split
import pandas as pd
data_path="/home/user/ksharma/ks_thesis/ma-thesis/pipeline_test/data_files/df_2000_rows.csv"
sjmm_data = pd.read_csv(data_path)
# sjmm_data['input'] = sjmm_data['input'].str.rstrip(' ->')
fine_tuning_sjmm_data = sjmm_data[['input', 'output']]
fine_tuning_sjmm_data['ind_skill'] = fine_tuning_sjmm_data['output'].str.findall(r'<SoftSkill>(.*?)</SoftSkill>')
pd.set_option('display.max_colwidth', 1000)
train_sjmm_data, test_sjmm_data = train_test_split(fine_tuning_sjmm_data, test_size=0.1, random_state=42)
test_sjmm_data

,input,output,ind_skill
752,"Wir wenden uns an einen gelernten Uhrmacher mit einem <SoftSkill_C>dienstleitungsorientierten</SoftSkill_C>, <SoftSkill_C>verantwortungsbewussten</SoftSkill_C> wie <SoftSkill>teamorientierten handeln</SoftSkill>. ->","Wir wenden uns an einen gelernten Uhrmacher mit einem <SoftSkill>dienstleitungsorientierten handeln</SoftSkill>, <SoftSkill>verantwortungsbewussten handeln</SoftSkill> wie <SoftSkill>teamorientierten handeln</SoftSkill>.","[dienstleitungsorientierten handeln, verantwortungsbewussten handeln, teamorientierten handeln]"
765,Ihre <SoftSkill>Arbeitsweise ist proaktiv</SoftSkill> mit <SoftSkill_C>hoher Dienstleistungs-</SoftSkill_C> und <SoftSkill>Einsatzbereitschaft</SoftSkill>. ->,Ihre <SoftSkill>Arbeitsweise ist proaktiv</SoftSkill> mit <SoftSkill>hoher Dienstleistungsbereitschaft</SoftSkill> und <SoftSkill>Einsatzbereitschaft</SoftSkill>.,"[Arbeitsweise ist proaktiv, hoher Dienstleistungsbereitschaft, Einsatzbereitschaft]"
1655,"Voraussetzungen sind eine abgeschlossene Volksschule (Sekundarschule A / B), <SoftSkill_C>psychische</SoftSkill_C> und <SoftSkill>physische Gesundheit</SoftSkill>, <SoftSkill>Kontaktfreudigkeit</SoftSkill>, eine <SoftSkill>gute Auffassungsgabe</SoftSkill>, <SoftSkill_C>Team-</SoftSkill_C> und <SoftSkill>Kommunikations-Fähigkeit</SoftSkill>, <SoftSkill>praktisches Verständnis</SoftSkill> und <SoftSkill>manuelles Geschick</SoftSkill>. ->","Voraussetzungen sind eine abgeschlossene Volksschule (Sekundarschule A / B), <SoftSkill>psychische Gesundheit</SoftSkill> und <SoftSkill>physische Gesundheit</SoftSkill>, <SoftSkill>Kontaktfreudigkeit</SoftSkill>, eine <SoftSkill>gute Auffassungsgabe</SoftSkill>, <SoftSkill>Teamfähigkeit</SoftSkill> und <SoftSkill>Kommunikationsfähigkeit</SoftSkill>, <SoftSkill>praktisches Verständnis</SoftSkill> und <SoftSkill>manuelles Geschick</SoftSkill>.","[psychische Gesundheit, physische Gesundheit, Kontaktfreudigkeit, gute Auffassungsgabe, Teamfähigkeit, Kommunikationsfähigkeit, praktisches Verständnis, manuelles Geschick]"
1252,Sie <SoftSkill_C>wertschätzen</SoftSkill_C> und <SoftSkill>respektieren die Erfahrung der Betroffenen und deren Angehörigen</SoftSkill>. ->,Sie <SoftSkill>wertschätzen die Erfahrung der Betroffenen und deren Angehörigen</SoftSkill> und <SoftSkill>respektieren die Erfahrung der Betroffenen und deren Angehörigen</SoftSkill>.,"[wertschätzen die Erfahrung der Betroffenen und deren Angehörigen, respektieren die Erfahrung der Betroffenen und deren Angehörigen]"
1102,"In unserer täglichen Arbeit legen wir grossen Wert darauf, <SoftSkill_C>wertschätzend</SoftSkill_C> und <SoftSkill>respektvoll miteinader zu kommunizieren</SoftSkill> und <SoftSkill>konstruktiv im Interesse einer hohen Patientenorientierung zusammen zu arbeiten</SoftSkill>. ->","In unserer täglichen Arbeit legen wir grossen Wert darauf, <SoftSkill>wertschätzend miteinader zu kommunizieren</SoftSkill> und <SoftSkill>respektvoll miteinader zu kommunizieren</SoftSkill> und <SoftSkill>konstruktiv im Interesse einer hohen Patientenorientierung zusammen zu arbeiten</SoftSkill>.","[wertschätzend miteinader zu kommunizieren, respektvoll miteinader zu kommunizieren, konstruktiv im Interesse einer hohen Patientenorientierung zusammen zu arbeiten]"
...,...,...,...
306,"Sie <SoftSkill>agieren in schwierigen Situationen ruhig</SoftSkill>, <SoftSkill_C>vertrauenswürdig</SoftSkill_C> und <SoftSkill_C>mit souveräner Umsicht</SoftSkill_C>. ->","Sie <SoftSkill>agieren in schwierigen Situationen ruhig</SoftSkill>, <SoftSkill>agieren vertrauenswürdig</SoftSkill> und <SoftSkill>agieren mit souveräner Umsicht</SoftSkill>.","[agieren in schwierigen Situationen ruhig, agieren vertrauenswürdig, agieren mit souveräner Umsicht]"
976,• <SoftSkill_C>Proaktives</SoftSkill_C> und <SoftSkill>vorausschauendes Denken</SoftSkill> und <SoftSkill_C>Handeln</SoftSkill_C ->,<SoftSkill>Proaktives Denken</SoftSkill> und <SoftSkill>vorausschauendes Denken</SoftSkill> und <SoftSkill>vorausschauend

In [25]:
predicted_values = []
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
for text in test_sjmm_data['input'][:3]:
    input_ids = tokenizer.encode(text, return_tensors="pt")
    input_ids = input_ids.to(device)
    # Generate the output text
    output_ids = model.generate(input_ids=input_ids, max_new_tokens=280)
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    predicted_values.append(output_text)
    # testing_df = test_sjmm_data.copy()
    # testing_df['generated_value'] = predicted_values    

In [26]:
predicted_values

['Wir wenden uns an einen gelernten Uhrmacher mit einem SoftSkill>dienstleitungsorientierten handeln/SoftSkill>, SoftSkill>verantwortungsbewussten handeln/SoftSkill> wie SoftSkill>teamorientierten handeln/SoftSkill>.',
 'Ihre SoftSkill>Arbeitsweise ist proaktiv/SoftSkill> mit SoftSkill>hoher Dienstleistungsbereitschaft/SoftSkill> und SoftSkill>Einsatzbereitschaft/SoftSkill>.',
 'Voraussetzungen sind eine abgeschlossene Volksschule (Sekundarschule A / B), SoftSkill>psychische Gesundheit/SoftSkill> und SoftSkill>physische Gesundheit/SoftSkill>, SoftSkill>Kontaktfreudigkeit/SoftSkill>, eine SoftSkill>gute Auffassungsgabe/SoftSkill>, SoftSkill>Teamfähigkeit/SoftSkill> und SoftSkill>Kommunikationsfähigkeit/SoftSkill>, SoftSkill>praktisches Verständnis/SoftSkill> und SoftSkill>manuelles Geschick/SoftSkill>.']

In [35]:
import pandas as pd 
data_path="/home/user/ksharma/ks_thesis/ma-thesis/pipeline_test/data_files/MASTER_TEST_SET.csv"
results_FLAN_T5_XXL = pd.read_csv(data_path)
results_FLAN_T5_XXL = results_FLAN_T5_XXL[results_FLAN_T5_XXL['valid'] == 1]
results_FLAN_T5_XXL['expected_skills'] = results_FLAN_T5_XXL['expected_text'].str.findall(r'<SoftSkill>(.*?)</SoftSkill>')

# Create a new column 'generated_output' to store the generated output text
results_FLAN_T5_XXL['generated_text'] = ''
results_FLAN_T5_XXL['generated_skills_all'] = ''

# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# model_pretrained.to(device)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Tokenize the input text and add the generated output text to the DataFrame
for index, row in tqdm(results_FLAN_T5_XXL.iterrows(), total=len(results_FLAN_T5_XXL), desc="Generating Text"):
# for index, row in test_set[:2].iterrows():
    input_text = row['input']
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    input_ids = input_ids.to(device)

    # Generate the output text
    output_ids = model.generate(input_ids=input_ids, max_new_tokens=280)
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Assign the generated output text to the corresponding row in the DataFrame
    results_FLAN_T5_XXL.at[index, 'generated_text'] = output_text

    gen_skills = re.findall(r'(?:SoftSkill>|SoftSkill_C>)(.*?)(?:/SoftSkill>|/SoftSkill_C>)', output_text)
    results_FLAN_T5_XXL.at[index, 'generated_skills_all'] = gen_skills

Generating Text: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 195/195 [28:17<00:00,  8.70s/it]


In [36]:
results_FLAN_T5_XXL

,orig_idx,input,expected_text,Problem type,expected_skills,valid,generated_text,generated_skills_all
0,752,"Wir wenden uns an einen gelernten Uhrmacher mit einem <SoftSkill_C>dienstleitungsorientierten</SoftSkill_C>, <SoftSkill_C>verantwortungsbewussten</SoftSkill_C> wie <SoftSkill>teamorientierten handeln</SoftSkill>.","Wir wenden uns an einen gelernten Uhrmacher mit einem <SoftSkill>dienstleitungsorientierten handeln</SoftSkill>, <SoftSkill>verantwortungsbewussten handeln</SoftSkill> wie <SoftSkill>teamorientierten handeln</SoftSkill>.",MC,"[dienstleitungsorientierten handeln, verantwortungsbewussten handeln, teamorientierten handeln]",1,"Wir wenden uns an einen gelernten Uhrmacher mit einem SoftSkill>dienstleitungsorientierten handeln/SoftSkill>, SoftSkill>verantwortungsbewussten handeln/SoftSkill> wie SoftSkill>teamorientierten handeln/SoftSkill>.","[dienstleitungsorientierten handeln, verantwortungsbewussten handeln, teamorientierten handeln]"
1,765,Ihre <SoftSkill>Arbeitsweise ist proaktiv</SoftSkill> mit <SoftSkill_C>hoher Dienstleistungs-</SoftSkill_C> und <SoftSkill>Einsatzbereitschaft</SoftSkill>.,Ihre <SoftSkill>Arbeitsweise ist proaktiv</SoftSkill> mit <SoftSkill>hoher Dienstleistungsbereitschaft</SoftSkill> und <SoftSkill>Einsatzbereitschaft</SoftSkill>.,HOWA,"[Arbeitsweise ist proaktiv, hoher Dienstleistungsbereitschaft, Einsatzbereitschaft]",1,Ihre SoftSkill>Arbeitsweise ist proaktiv/SoftSkill> mit SoftSkill>hoher Dienstleistungsbereitschaft/SoftSkill> und SoftSkill>Einsatzbereitschaft/SoftSkill>.,"[Arbeitsweise ist proaktiv, hoher Dienstleistungsbereitschaft, Einsatzbereitschaft]"
2,1655,"Voraussetzungen sind eine abgeschlossene Volksschule (Sekundarschule A / B), <SoftSkill_C>psychische</SoftSkill_C> und <SoftSkill>physische Gesundheit</SoftSkill>, <SoftSkill>Kontaktfreudigkeit</SoftSkill>, eine <SoftSkill>gute Auffassungsgabe</SoftSkill>, <SoftSkill_C>Team-</SoftSkill_C> und <SoftSkill>Kommunikations-Fähigkeit</SoftSkill>, <SoftSkill>praktisches Verständnis</SoftSkill> und <SoftSkill>manuelles Geschick</SoftSkill>.","Voraussetzungen sind eine abgeschlossene Volksschule (Sekundarschule A / B), <SoftSkill>psychische Gesundheit</SoftSkill> und <SoftSkill>physische Gesundheit</SoftSkill>, <SoftSkill>Kontaktfreudigkeit</SoftSkill>, eine <SoftSkill>gute Auffassungsgabe</SoftSkill>, <SoftSkill>Teamfähigkeit</SoftSkill> und <SoftSkill>Kommunikationsfähigkeit</SoftSkill>, <SoftSkill>praktisches Verständnis</SoftSkill> und <SoftSkill>manuelles Geschick</SoftSkill>.",HMDC,"[psychische Gesundheit, physische Gesundheit, Kontaktfreudigkeit, gute Auffassungsgabe, Teamfähigkeit, Kommunikationsfähigkeit, praktisches Verständnis, manuelles Geschick]",1,"Voraussetzungen sind eine abgeschlossene Volksschule (Sekundarschule A / B), SoftSkill>psychische Gesundheit/SoftSkill> und SoftSkill>physische Gesundheit/SoftSkill>, SoftSkill>Kontaktfreudigkeit/SoftSkill>, eine SoftSkill>gute Auffassungsgabe/SoftSkill>, SoftSkill>Teamfähigkeit/SoftSkill> und SoftSkill>Kommunikationsfähigkeit/SoftSkill>, SoftSkill>praktisches Verständnis/SoftSkill> und SoftSkill>manuelles Geschick/SoftSkill>.","[psychische Gesundheit, physische Gesundheit, Kontaktfreudigkeit, gute Auffassungsgabe, Teamfähigkeit, Kommunikationsfähigkeit, praktisches Verständnis, manuelles Geschick]"
3,1252,Sie <SoftSkill_C>wertschätzen</SoftSkill_C> und <SoftSkill>respektieren die Erfahrung der Betroffenen und deren Angehörigen</SoftSkill>.,Sie <SoftSkill>wertschätzen die Erfahrung der Betroffenen und deren Angehörigen</SoftSkill> und <SoftSkill>respektieren die Erfahrung der Betroffenen und deren Angehörigen</SoftSkill>.,MWA,"[wertschätzen die Erfahrung der Betroffenen und deren Angehörigen, respektieren die Erfahrung der Betroffenen und deren Angehörigen]",1,Sie SoftSkill>wertschätzen die Erfahrung der Betroffenen und deren Angehörigen/SoftSkill> und SoftSkill>respektieren die Erfahrung der Betroffenen und deren Angehörigen/SoftSkill>.,"[wertschätzen die Erfahrung der Betroff

In [38]:
results_FLAN_T5_XXL.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/FLAN_T5_XXL/model_generation.csv")

: 